In [2]:
import pandas as pd
import os
from bs4 import BeautifulSoup
import re, requests
import time
import numpy as np

class DotaLatestInfo:
    hero_df=None
    df_advantage=None
    df_disadvantage=None
    df_lane_position=None
    df_item_hero=None
    def __init__(self):
        self.push_info()
    def push_info(self):
        

        print('This will take about 5 minutes!!')
        
        
        dota_url='https://www.dotabuff.com/heroes/meta'
        r   = requests.get(dota_url, headers={'user-agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(r.content)
        table = soup('table')
        
        self.hero_df = pd.read_html(str(table))[0]
        del self.hero_df['Hero']
        #rename columns
        self.hero_df.columns = ['Hero', 'Pick%_Crusader', 'Win%_Crusader', 'Pick%_Crusader_Archon','Win%_Crusader_Archon',
                          'Pick%_Legend','Win%_Legend','Pick%Ancient','Win%Ancient','Pick%Divine_Imortal','Win%Divine_Imortal']
        
        #all of our columns are objects, we have to remove the '%' character and then transform it to numerical value
        for y in self.hero_df.columns:
            if y !='Hero':
                for w in range(len(self.hero_df[y])):
                    self.hero_df[y][w] = self.hero_df[y][w].translate({ord(i):None for i in '%'})
                    self.hero_df[y][w]=pd.to_numeric(self.hero_df[y][w])
        self.hero_df.reset_index(inplace=True)

        
        #Now we need to create  our table with the info about witch heroes are best against and worst against
        #Since we are dealing with 122 heroes, i would need 121 html to get the info
        # this is a tedious task to get one by one. So first we need to create a list with all of our heroes:
        #We will create in a manner that, even if the heroes meta change, we still be abe to use this script
        #Creating the list...
        hero_name = self.hero_df['Hero']

        #If we take a look, the html for each hero is like that: <https://www.dotabuff.com/heroes/HERO-NAME>
        # letters should be all in lower case, instead of space between the name it should be "-". Let's change our hero_name object:
        hero_name_new = []
        for i in range(len(hero_name)):
            if "-" in hero_name[i]:
                hero_name_new.append(hero_name[i].lower())
            elif "'" in hero_name[i]:
                hero_name_new.append(hero_name[i].lower().replace("'","").replace(' ','-'))
            else:
                hero_name_new.append(hero_name[i].lower().replace(' ',"-"))
        
        #Now we can create several soups for our heroes
        #First, we are going to create a list of urls:
        heroes_url = []
        for i in range(len(hero_name_new)):
            heroes_url.append('https://www.dotabuff.com/heroes/'+hero_name_new[i])
        
        #Now let's create our requests and our soups. We are going to use time.sleep function, since we are not wanting to make a ddos attack:
        requests_hero   = []

        for i in range(len(heroes_url)):
            time.sleep(0.1)
            requests_hero.append(requests.get(heroes_url[i], headers={'user-agent': 'Mozilla/5.0'}))
        
        #Ok, now let's create our soups!
        soups_heroes =[] 
        for i in range(len(requests_hero)):
            soups_heroes.append(BeautifulSoup(requests_hero[i].content))
        list_advantage=[]
        for i in range(len(soups_heroes)):

            a = pd.read_html(str(soups_heroes[i]('table')))[3]
            a['MetaHero'] = self.hero_df['Hero'][i]
            del a['Hero']
            a.rename(columns = {'Hero.1':'Better against'},inplace = True)
            a['MetaHero'] = self.hero_df['Hero'][i]
            list_advantage.append(a)
            self.df_advantage = pd.concat(list_advantage)
            self.df_advantage.reset_index(drop=True,inplace=True)
            for w in self.df_advantage.columns:
                if ((w!='Better against') and(w!='Matches') and(w!='MetaHero')):
                    for z in range(len(self.df_advantage.index)):
                        s =  self.df_advantage.at[z,w].replace('%','')
                        self.df_advantage.at[z,w] = pd.to_numeric(s)


        list_disadvantage=[]
        for i in range(len(soups_heroes)):

            a = pd.read_html(str(soups_heroes[i]('table')))[4]
            a['MetaHero'] = self.hero_df['Hero'][i]
            del a['Hero']
            a.rename(columns = {'Hero.1':'Worst against'},inplace = True)

            list_disadvantage.append(a)
            self.df_disadvantage = pd.concat(list_disadvantage)
            self.df_disadvantage.reset_index(drop=True,inplace=True)
            for w in self.df_disadvantage.columns:
                if ((w!='Worst against') and(w!='Matches') and(w!='MetaHero')):
                    for z in range(len(self.df_disadvantage.index)):
                        s =  self.df_disadvantage.at[z,w].replace('%','')
                        self.df_disadvantage.at[z,w] = pd.to_numeric(s)

        list_lane_position=[]
        for i in range(len(soups_heroes)):
            a = pd.read_html(str(soups_heroes[i]('table')))[1]
            a['MetaHero'] = self.hero_df['Hero'][i]    
            list_lane_position.append(a)
            self.df_lane_position = pd.concat(list_lane_position)
            self.df_lane_position.reset_index(drop=True,inplace=True)
            for w in self.df_lane_position.columns:
                if ((w!='Lane')and(w!='KDA Ratio')and(w!='GPM')and(w!='XPM')and(w!='MetaHero')):
                    for z in range(len(self.df_lane_position.index)):
                        s =  self.df_lane_position.at[z,w].replace('%','')
                        self.df_lane_position.at[z,w] = pd.to_numeric(s)

        list_item_hero=[]
        for i in range(len(soups_heroes)):
            a = pd.read_html(str(soups_heroes[i]('table')))[2]
            a['MetaHero'] = self.hero_df['Hero'][i]
            del a['Item']
            a.rename(columns = {'Item.1':'Item'},inplace = True)
            list_item_hero.append(a)
            self.df_item_hero = pd.concat(list_item_hero)
            self.df_item_hero.reset_index(drop=True,inplace=True)
            for w in self.df_item_hero.columns:
                if ((w!='Item')and(w!='Matches')and(w!='Wins')and(w!='MetaHero')):
                    for z in range(len(self.df_item_hero.index)):
                        s =  self.df_item_hero.at[z,w].replace('%','')
                        self.df_item_hero.at[z,w] = pd.to_numeric(s)

    def get_top5(self,by,elo):
        if ((by =='Pick')and(elo=='Crusader')):
            return self.hero_df.groupby(['Pick%_Crusader','Hero','Win%_Crusader']).sum().sort_values('Pick%_Crusader',ascending=False).head(5)

        elif((by=='Pick')and(elo=='Crusader_Archon')):
            return self.hero_df.groupby(['Pick%_Crusader_Archon','Hero','Win%_Crusader_Archon']).sum().sort_values('Pick%_Crusader_Archon',ascending=False).head(5)

        elif((by=='Pick')and(elo=='Legend')):
            return self.hero_df.groupby(['Pick%_Legend','Hero','Win%_Legend']).sum().sort_values('Pick%_Legend',ascending=False).head(5)

        elif((by=='Pick')and (elo=='Ancient')):
            return self.hero_df.groupby(['Pick%Ancient','Hero','Win%_Ancient']).sum().sort_values('Pick%Ancient',ascending=False).head(5)

        elif((by=='Pick')and(elo=='Divine_Imortal')):
            return self.hero_df.groupby(['Pick%Divine_Imortal','Hero','Win%Divine_Imortal']).sum().sort_values('Pick%Divine_Imortal',ascending=False).head(5)

        elif((by=='Win')and(elo=='Crusader')):
            return self.hero_df.groupby(['Win%_Crusader','Hero','Pick%_Crusader']).sum().sort_values('Win%_Crusader',ascending=False).head(5)

        elif((by=='Win')and(elo=='Crusader_Archon')):
            return self.hero_df.groupby(['Win%_Crusader_Archon','Hero','Pick%_Crusader_Archon']).sum().sort_values('Win%_Crusader_Archon',ascending=False).head(5)

        elif((by=='Win')and(elo=='Legend')):
            return self.hero_df.groupby(['Win%_Legend','Hero','Pick%_Legend']).sum().sort_values('Win%_Legend',ascending=False).head(5)

        elif((by=='Win')and(elo=='Ancient')):
            return self.hero_df.groupby(['Win%Ancient','Hero','Pick%Ancient']).sum().sort_values('Win%Ancient',ascending=False).head(5)

        elif((by=='Win')and(elo=='Divine_Imortal')):
            return self.hero_df.groupby(['Win%Divine_Imortal','Hero','Pick%Divine_Imortal']).sum().sort_values('Win%Divine_Imortal',ascending=False).head(5)
        
    def get_advantage_disadvantage(self,hero):
        my_list=[]
        for i in self.hero_df['Hero']:
            for w in hero:
                if w ==i:
                    for w in hero:
                        a=self.df_advantage[self.df_advantage['MetaHero']==w].join(self.df_disadvantage[self.df_disadvantage['MetaHero']==w],
                                                                         how='left',lsuffix='_Advantage',rsuffix='_Disadvantage')
                        my_list.append(a)
                        my_df=pd.concat(my_list)
                    return my_df
                
    def get_lane_position(self,by):
        my_list=[]
        for i in self.df_lane_position['MetaHero'].unique():
            for w in by:
                if w==i:
                    a=self.df_lane_position[self.df_lane_position['MetaHero']==w].groupby(['Presence','Win Rate','KDA Ratio','GPM','XPM','MetaHero']).sum().sort_values('Win Rate',ascending=False)
                    my_list.append(a)

                    my_df=pd.concat(my_list)
        return my_df
    
    def get_item_hero(self,hero):
        my_list=[]
        for i in self.hero_df['Hero']:
            for w in hero:
                if w==i:
                    a = self.df_item_hero[self.df_item_hero['MetaHero']==w].groupby(['Win Rate','Item','MetaHero']).sum().sort_values('Win Rate',ascending=False)
                    my_list.append(a)
                    my_df=pd.concat(my_list)
        return my_df
    
    def item_info(self,item):
    
        item_name=[]
        for i in range(len(item)):

            if "'" in item[i]:
                item_name.append(item[i].lower().replace("'","").replace(' ','-'))
            else:
                item_name.append(item[i].lower().replace(' ','-'))

        items_url = []
        for i in range(len(item_name)):
            items_url.append('https://www.dotabuff.com/items/'+item_name[i])


        requests_item   = []

        for i in range(len(items_url)):

            requests_item.append(requests.get(items_url[i], headers={'user-agent': 'Mozilla/5.0'}))

        soups_items =[] 
        for i in range(len(requests_item)):
            soups_items.append(BeautifulSoup(requests_item[i].content))

        for w in range(len(soups_items)):
            print('\033[1m' + soups_items[w].find("div",class_='name').text + '\033[0m','\n',soups_items[w].find("div",class_='stats').text)
            print(soups_items[w].find("div",class_='description').text)

    